## Calculation of Merkle Root and Proofs for Final Distribution

In [8]:
import pandas as pd
import os
from multiprocess import Pool
from math import ceil
import json
from IPython.core.display import display, HTML

from contract_utils import instantiate_contract, execute_contract, query_contract, get_proofs

INITIAL_BALANCE = str(10_000_000_000)
NUMBER_OF_ACTIVATED_PARTICIPANTS = str(10)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(10)

NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
DISPLAY_TX_EXECUTION = False

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

CALCULATE_PROOFS = True
STATE_FILE_NAME = 'gift_state_final_220320.csv'
ROOT_SOURCE_FILE_NAME = 'root_source_final.json'
PROOF_FILE_NAME = 'proof_final.json'
ROOT_FILE_NAME = 'root_final'

In [2]:
state_df = pd.read_csv(STATE_FILE_NAME)
display(HTML(state_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

### Create Merkle Tree, Get Root

In [4]:
state_agg_by_address = state_df.groupby('address')['gift'].sum().reset_index().rename(columns={'gift': 'amount'})
state_agg_by_address['amount'] = state_agg_by_address['amount'] * 1_000_000

In [5]:
root_source_list = state_agg_by_address.to_dict(orient='records')
number_of_addresses = len(root_source_list)
print(f'Number of addresses: {number_of_addresses}')

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

Number of addresses: 4426091


### Calculate Merkle Root and Proofs

In [10]:
if CALCULATE_PROOFS:
    tasks = set(
        (
            ('root_testing_source_1000_addresses.json',
             f'temp/proofs_{i}.json',
             i * BASH_SIZE + 1,
             min(number_of_addresses + 1, (i + 1) * BASH_SIZE + 1))
            for i in range(ceil(number_of_addresses/BASH_SIZE))
        )
    )
    print(tasks)
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tasks)
    assert res == [True] * len(res)

In [7]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
root = roots[0]
print(f'Merkle root: {root}')
print(f'Number of proofs: {len(proofs_df)}')
display(HTML(proofs_df.head().to_html(index=False, notebook=True, show_dimensions=False)))
proofs_df.to_csv('proofs_final.csv', header=True, index=True)

''

## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [ ]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

### Instantiate Passport Contract

In [ ]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

### Set executor in the Subgraph Contracts

In [25]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [ ]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

## Register Merkle Root

In [ ]:
root_register_output = execute_contract(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        contract_address=gift_contract_address)

In [ ]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))
